In [12]:
import tensorflow as tf

import os

# Just disables the warning, doesn't enable AVX/FMA
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [13]:
import numpy as np
import pandas as pd

In [14]:
census = pd.read_csv("/Users/rajacsp/datasets/census_data.csv")

In [15]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [18]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [19]:
# Fix the income bracket
def label_fix(label):
    label = label.strip().lower()
    if(label == '<=50k'):
        return 0
    return 1
        

In [20]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [21]:
census['income_bracket'].unique()

array([0, 1])

In [25]:
# Train Test Split
from sklearn.model_selection import train_test_split



In [27]:
x_data = census.drop('income_bracket', axis=1)
y_labels = census['income_bracket']

X_train, x_test, y_train, y_est = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [28]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [32]:
import tensorflow as tf
from tensorflow.feature_column import categorical_column_with_vocabulary_list as vlist

In [33]:
gender = vlist("gender", ["Female", "Male"])

In [34]:
gender

VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [35]:
from tensorflow.feature_column import categorical_column_with_hash_bucket as bucket

In [38]:
occupation = bucket("occupation", hash_bucket_size=1000)
marital_status = bucket("marital_status", hash_bucket_size=1000)
relationship = bucket("relationship", hash_bucket_size=1000)
education = bucket("education", hash_bucket_size=1000)
workclass = bucket("workclass", hash_bucket_size=1000)
native_country = bucket("native_country", hash_bucket_size=1000)

In [39]:
from tensorflow.feature_column import numeric_column as nc

In [41]:
age = nc("age")
education_num = nc("education_num")
capital_gain = nc("capital_gain")
capital_loss = nc("capital_loss")
hours_per_week = nc("hours_per_week")

In [42]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country, age, education_num, 
            capital_gain, capital_loss, hours_per_week]

In [43]:
feat_cols

[VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 HashedCategoricalColumn(key='occupation', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='marital_status', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='relationship', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='education', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='workclass', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='native_country', hash_bucket_size=1000, dtype=tf.string),
 NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='capital_loss', shape

In [45]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y = y_train, batch_size=100, num_epochs=None, shuffle=True)

In [46]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/fk/b_m_3t9915zc2llvfkwd4d_h0000gn/T/tmp3xkmlm9h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x130e60e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
model.train(input_fn=input_func, steps=5000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/fk/b_m_3t9915zc2llvfkwd4d_h0000gn/T/tmp3xkmlm9h/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 183.162
INFO:tensorflow:loss = 74.34316, step = 101 (0.549 sec)
INFO:tensorflow:global_step/sec: 282.272
INFO:tensorflow:loss = 258.81635, step = 201 (0

In [50]:
# Evaluation

pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test, batch_size=len(x_test), shuffle=False)

In [51]:
prediction = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/fk/b_m_3t9915zc2llvfkwd4d_h0000gn/T/tmp3xkmlm9h/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [53]:
prediction[0]

{'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.24731174], dtype=float32),
 'logits': array([-1.1130015], dtype=float32),
 'probabilities': array([0.7526882 , 0.24731174], dtype=float32)}

In [55]:
final_preds = []

for pred in prediction:
    final_preds.append(pred['class_ids'][0])

In [56]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [57]:
from sklearn.metrics import classification_report

In [59]:
print(classification_report(y_est, final_preds))

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      7436
           1       0.70      0.54      0.61      2333

   micro avg       0.83      0.83      0.83      9769
   macro avg       0.78      0.73      0.75      9769
weighted avg       0.83      0.83      0.83      9769

